In [1]:
import cv2 as cv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd



In [2]:
%matplotlib qt

Failed to create wl_display (No such file or directory)
qt.qpa.plugin: Could not load the Qt platform plugin "wayland" in "/home/felipe/Python/.venv/lib/python3.12/site-packages/cv2/qt/plugins" even though it was found.


In [59]:
#Armazena a imagem na variavel 'img'
img= cv.imread('Data/PP degradado/image10.tif')

In [60]:
# Defina aqui o valor do intervalo em micrômetros correspondente ao segmento selecionado
intervalo_um = 1000
  # Exemplo: 100 μm (ajuste aqui conforme sua escala real)

# Abre a janela para selecionar dois pontos
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.title('definindo a escala')
pontos = plt.ginput(2)
plt.close()
plt.show()

# Calcula o intervalo em pixels para sua escala
(x1, y1), (x2, y2) = pontos
intervalo_pixels = ((x2 - x1)**2 + (y2 - y1)**2)**0.5

# Fator de conversão: quantos μm por pixel
pixel_para_um = intervalo_um / intervalo_pixels

print(f'{intervalo_um} μm = {intervalo_pixels:.2f} pixels')
print(f'1 pixel = {pixel_para_um:.4f} μm')

# ...restante do código...
# Para área:
#areas_um2 = [area * (pixel_para_um ** 2) for area in areas_pixels]

1000 μm = 169.36 pixels
1 pixel = 5.9047 μm


In [61]:
# mostra a imagem para selecionar uma area de interesse
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.title('Selecione a area de interesse')
roi_points = plt.ginput(2)
plt.close()
plt.show()

#extrai as coordenada da roi

(x1, y1), (x2, y2) = roi_points
x1, x2 = int(round(x1)), int(round(x2))
y1, y2 = int(round(y1)), int(round(y2))

#indices em ordem
xmin, xmax = sorted([x1, x2])
ymin, ymax = sorted([y1, y2])

# recorta a região de interesse
roi = img[ymin:ymax, xmin:xmax]

# aumenta a resolução da ROI (ex: dobra o tamanho)
escala = 2  # ou 3, conforme necessário
roi_highres = cv.resize(roi, None, fx=escala, fy=escala, interpolation=cv.INTER_CUBIC)

# exibe região selecionada com maior resolução
plt.imshow(cv.cvtColor(roi_highres, cv.COLOR_BGR2RGB))
plt.title('Região de Interesse (alta resolução)')
plt.show()


In [62]:
# 1. Pré-processamento: converte para escala de cinza e aplica blur
roi_gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
roi_eq = cv.equalizeHist(roi_gray)
roi_blur = cv.GaussianBlur(roi_eq, (5, 5), 0)

# 2. Binarização (ajuste o valor do threshold conforme necessário)
thresh_value = 127  # ou outro valor que funcione bem para sua imagem
_, roi_bin = cv.threshold(roi_gray, thresh_value, 255, cv.THRESH_TOZERO)

# Dilatação leve
kernel = np.ones((2,2), np.uint8)
roi_bin = cv.dilate(roi_bin, kernel, iterations=1)


plt.imshow(roi_bin, cmap='gray')
plt.title('Binarização Invertida')
plt.axis('off')
plt.savefig('/home/felipe/Python/Kasparov Project/Validação/Valdenilson/PP degradado/Trashold10.jpeg', dpi=300)
plt.show()



In [63]:
# 3. Detecção de contornos
contornos, _ = cv.findContours(roi_bin, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Crie uma cópia da ROI para desenhar os contornos
roi_contorno = roi.copy()
cv.drawContours(roi_contorno, contornos, -1, (0,255,0), 2)  # Verde, espessura 2

# Desenhar o número de cada contorno no centroide
for i, cnt in enumerate(contornos):
    M = cv.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        # Desenha o número no centro do fragmento
        cv.putText(roi_contorno, str(i+1), (cx, cy),
                   cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)


plt.figure(figsize=(8,8))
plt.imshow(cv.cvtColor(roi_contorno, cv.COLOR_BGR2RGB))
plt.title('Áreas contornadas')
plt.axis('off')
plt.savefig('/home/felipe/Python/Kasparov Project/Validação/Valdenilson/PP degradado/Contagem10.jpeg', dpi=300)

plt.show()

# 4. Calcula área de cada contorno em pixels e em μm²
pixel_para_um = intervalo_um / intervalo_pixels
areas_pixels = [cv.contourArea(cnt) for cnt in contornos]
areas_um2 = [area * (pixel_para_um ** 2) for area in areas_pixels]

dados = []  # Lista para armazenar os dados de cada fragmento

print(f'Elementos detectados: {len(contornos)}')
for i, (cnt, area_px, area_um) in enumerate(zip(contornos, areas_pixels, areas_um2)):
    # Calcula o centroide do contorno
    M = cv.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        if 0 <= cy < roi.shape[0] and 0 <= cx < roi.shape[1]:
            rgb_centro = roi[cy, cx]
        else:
            rgb_centro = [0, 0, 0]
    else:
        rgb_centro = [0, 0, 0]

    # Salva os dados em um dicionário
    dados.append({
        'elemento': i+1,
        'area_pixels': area_px,
        'area_um2': area_um,
        'rgb_centro': rgb_centro
    })

    print(
        f'Elemento {i+1}: área = {area_px:.2f} px² | {area_um:.2f} μm² | '
        f'RGB do centro = {np.array(rgb_centro).astype(int)}'
    )

# Cria o DataFrame
df_fragmentos = pd.DataFrame(dados)

# Se quiser separar o RGB em colunas distintas:
df_fragmentos[['R', 'G', 'B']] = pd.DataFrame(df_fragmentos['rgb_centro'].tolist(), index=df_fragmentos.index)
df_fragmentos = df_fragmentos.drop(columns=['rgb_centro'])

df_fragmentos = df_fragmentos[df_fragmentos['area_um2'] != 0].reset_index(drop=True)

# Exibe o DataFrame
#df_fragmentos

Elementos detectados: 976
Elemento 1: área = 0.00 px² | 0.00 μm² | RGB do centro = [0 0 0]
Elemento 2: área = 0.00 px² | 0.00 μm² | RGB do centro = [0 0 0]
Elemento 3: área = 0.00 px² | 0.00 μm² | RGB do centro = [0 0 0]
Elemento 4: área = 10.00 px² | 348.65 μm² | RGB do centro = [159 136 127]
Elemento 5: área = 6.50 px² | 226.62 μm² | RGB do centro = [112  95  98]
Elemento 6: área = 18.00 px² | 627.57 μm² | RGB do centro = [115 164 168]
Elemento 7: área = 8.50 px² | 296.35 μm² | RGB do centro = [165 144 123]
Elemento 8: área = 8.00 px² | 278.92 μm² | RGB do centro = [191 171 161]
Elemento 9: área = 8.50 px² | 296.35 μm² | RGB do centro = [190 176 178]
Elemento 10: área = 87.00 px² | 3033.28 μm² | RGB do centro = [224 226 231]
Elemento 11: área = 15.50 px² | 540.41 μm² | RGB do centro = [151 158 156]
Elemento 12: área = 2.00 px² | 69.73 μm² | RGB do centro = [149 131 131]
Elemento 13: área = 12.00 px² | 418.38 μm² | RGB do centro = [166 147 137]
Elemento 14: área = 28.00 px² | 976.23 μ

In [64]:
# ...existing code...

dados = []  # Lista para armazenar os dados de cada fragmento

print(f'Elementos detectados: {len(contornos)}')
for i, (cnt, area_px, area_um) in enumerate(zip(contornos, areas_pixels, areas_um2)):
    # Coleta as cores RGB do centro (máscara preenchida)
    mask = np.zeros(roi.shape[:2], dtype=np.uint8)
    cv.drawContours(mask, [cnt], -1, 255, thickness=cv.FILLED)
    pixels_centro = roi[mask == 255]
    if len(pixels_centro) > 0:
        media_rgb_centro = pixels_centro.mean(axis=0)
    else:
        media_rgb_centro = [0, 0, 0]

    # Salva os dados em um dicionário
    dados.append({
        'elemento': i+1,
        'area_pixels': area_px,
        'area_um2': area_um,
        'rgb_medio': media_rgb_centro
    })

    #print(
     #   f'Elemento {i+1}: área = {area_px:.2f} px² | {area_um:.2f} μm² | '
      #  f'RGB médio do centro = {np.array(media_rgb_centro).astype(int)}'
   # )

# Cria o DataFrame
df_fragmentos = pd.DataFrame(dados)

# Se quiser separar o RGB em colunas distintas:
df_fragmentos[['R', 'G', 'B']] = pd.DataFrame(df_fragmentos['rgb_medio'].tolist(), index=df_fragmentos.index)
df_fragmentos = df_fragmentos.drop(columns=['rgb_medio'])

df_fragmentos = df_fragmentos[df_fragmentos['area_um2'] != 0].reset_index(drop=True)

# Exibe o DataFrame
#df_fragmentos
df_fragmentos.to_csv('/home/felipe/Python/Kasparov Project/Validação/Valdenilson/PP degradado/Fragmentos10.csv', index=False)

Elementos detectados: 976


In [65]:
# Cálculo do IQR para filtrar outliers
q1 = df_fragmentos['area_um2'].quantile(0.25)
q3 = df_fragmentos['area_um2'].quantile(0.75)
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

# Filtra os dados sem outliers
area_filtrada = df_fragmentos[(df_fragmentos['area_um2'] >= limite_inferior) & 
                              (df_fragmentos['area_um2'] <= limite_superior)]['area_um2']



bin_width = 10
bins = np.arange(0, area_filtrada.max() + bin_width, bin_width)

plt.figure(figsize=(8,5))
plt.hist(area_filtrada, bins=bins, color='darkred', edgecolor='black')
plt.xlabel('Área (μm²)')
plt.ylabel('Contagem absoluta')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.yticks(np.arange(0, plt.gca().get_ylim()[1] + 10, 10))
plt.title('Distribuição dos fragmentos (sem outliers)')

# Ticks do eixo x a cada 5 bins (ou seja, 250 unidades)
#xticks = bins[::5]
#if bins[-1] not in xticks:
#    xticks = np.append(xticks, bins[-1])
#plt.xticks(xticks, rotation=90)
plt.xlim(0,250)
plt.xticks(np.arange(0,250,10), rotation=90)
plt.tight_layout()
plt.savefig('/home/felipe/Python/Kasparov Project/Validação/Valdenilson/PP degradado/Distribuicao10.jpeg', dpi=300)
plt.show()

In [ ]:
# Use o RGB do centro já salvo no DataFrame
rgb_centros = df_fragmentos[['R', 'G', 'B']].to_numpy()

# Converta para inteiro e normalize para [0, 1]
rgb_centros = np.clip(rgb_centros, 0, 255).astype(np.uint8) / 255.0

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_zlim(0, 1)

# O parâmetro 'c' recebe uma lista de cores RGB normalizadas
ax.scatter(
    rgb_centros[:,0], rgb_centros[:,1], rgb_centros[:,2],
    c=rgb_centros, s=80, edgecolor='k'
)

ax.set_xlabel('R')
ax.set_ylabel('G')
ax.set_zlabel('B')
ax.set_title('Espaço de cores RGB dos fragmentos')
plt.show()